In [0]:
!pip install dotenv

In [0]:
# %restart_python

In [0]:
dbutils.library.restartPython()

In [0]:
# DBTITLE 1,Set env
dbutils.widgets.dropdown('env', 'dev', ['dev', 'staging', 'prod'], 'Environment Name')

In [0]:
from typing import Dict

from booking.utils.notebook_utils import load_and_set_env_vars, load_config
from booking.model_inference import ModelInference
from booking.utils.logger_utils import get_logger

In [0]:
_logger = get_logger()

In [0]:
pipeline_name = 'model_inference_batch'

# Load pipeline config yaml file (../conf/pipeline_configs/{pipeline_name}.yml)
pipeline_config = load_config(pipeline_name)

# Load and set arbitrary params via spark_env_vars
# Params passed via ../conf/{env}/.{env}.env and ../conf/.base_data_params
env_vars = load_and_set_env_vars(env=dbutils.widgets.get('env'))

In [0]:
model_name = env_vars['model_name']
model_registry_stage = pipeline_config['mlflow_params']['model_registry_stage']
model_uri = f'models:/{model_name}/{model_registry_stage}'
print(f'model_uri: {model_uri}')

In [0]:
input_table_name = pipeline_config['data_input']['table_name']
print(f'input_table_name: {input_table_name}')

In [0]:
predictions_table_database_name = env_vars['predictions_table_database_name']
predictions_table_name = f'{predictions_table_database_name}.{env_vars["predictions_table_name"]}'
lookup_keys = env_vars["feature_store_table_primary_keys"]
feature_table_name= f'{env_vars["feature_store_database_name"]}.{env_vars["feature_store_table_name"]}'
print(f'predictions_table_name: {predictions_table_name}')

In [0]:
model_inference_pipeline = ModelInference(model_uri=model_uri,
                                          input_table_name=input_table_name,
                                          output_table_name=predictions_table_name,
                                          lookup_keys=lookup_keys,
                                          feature_table_name=feature_table_name)

model_inference_pipeline.run_and_write_batch(mode=pipeline_config['data_output']['mode'])